In [1]:
import numpy as np
import math
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals import joblib
%matplotlib inline

In [2]:
motions = ['nothing', 'unarmed-attack-r3']
data_files = {motion: 'data/{}.txt'.format(motion) for motion in motions}

In [3]:
# Load training data
data = {motion: np.loadtxt(file_name) for motion, file_name in data_files.items()}

In [4]:
# Statistics about training data
for motion in motions:
    print("{}: ".format(motion))
    print('    shape: {}'.format(data[motion].shape))

nothing: 
    shape: (109, 1062)
unarmed-attack-r3: 
    shape: (115, 1086)


### Preprocess

In [5]:
def regularize_shape(data, num_samples):
    '''
    return num_samples of entries in data,
    (Note: data are groups of 6dof measurements)
    '''
    data = data.reshape(data.shape[0], -1, 6)
    length = data.shape[1]
    assert(num_samples <= length)
    # step = math.ceil(data.shape[1]//num_samples)
    result = []
    for data_instance in data:
        result_instance = []
        for i in range(num_samples):
            result_instance.append(data_instance[int(math.ceil(i*length/num_samples))])
        result.append(np.asarray(result_instance))
    return np.asarray(result)
NUM_SAMPLES = 64
data_regularized = {key: regularize_shape(val, NUM_SAMPLES) for key, val in data.items()}
for motion in motions:
    print("{}: ".format(motion))
    print('    regularized shape: {}'.format(data_regularized[motion].shape))

nothing: 
    regularized shape: (109, 64, 6)
unarmed-attack-r3: 
    regularized shape: (115, 64, 6)


In [6]:
# Return X and y
def raw_data(data_dict, keys):
    lengths = [data_dict[key].shape[0] for key in keys]
    data_y = []
    for i in range(len(lengths)):
        data_y.extend([i]*lengths[i])
    data_y = np.asarray(data_y).reshape(-1, 1)
    data_X = np.concatenate([data_dict[key] for key in keys])
    assert(data_y.shape[0] == data_X.shape[0])
    return data_X, data_y
data_X, data_y = raw_data(data_regularized, motions)

In [7]:
def flatten(X):
    return X.reshape(X.shape[0], -1)
data_X_flattened = flatten(data_X)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data_X_flattened, data_y, test_size=0.1, random_state=0)

### Neural Network Model

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [10]:
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [25]:
model = Sequential()
model.add(Dense(4, input_shape=X_train.shape[1:], activation='relu'))
model.add(Dense(len(motions), activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 4)                 1540      
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 10        
Total params: 1,550
Trainable params: 1,550
Non-trainable params: 0
_________________________________________________________________


In [35]:
validation_split = 0.1
history = model.fit(X_train, y_train_categorical, batch_size=X_train.shape[0], epochs=40, verbose=1,
          validation_split=validation_split, shuffle=True)

Train on 180 samples, validate on 21 samples
Epoch 1/40
180/180 [==============================] - 0s 24us/step - loss: 0.0336 - acc: 0.9833 - val_loss: 0.3068 - val_acc: 0.9048
Epoch 2/40
180/180 [==============================] - 0s 25us/step - loss: 0.0335 - acc: 0.9833 - val_loss: 0.3072 - val_acc: 0.9048
Epoch 3/40
180/180 [==============================] - 0s 23us/step - loss: 0.0335 - acc: 0.9833 - val_loss: 0.3077 - val_acc: 0.9048
Epoch 4/40
180/180 [==============================] - 0s 23us/step - loss: 0.0334 - acc: 0.9833 - val_loss: 0.3081 - val_acc: 0.9048
Epoch 5/40
180/180 [==============================] - 0s 32us/step - loss: 0.0334 - acc: 0.9833 - val_loss: 0.3086 - val_acc: 0.9048
Epoch 6/40
180/180 [==============================] - 0s 23us/step - loss: 0.0334 - acc: 0.9833 - val_loss: 0.3090 - val_acc: 0.9048
Epoch 7/40
180/180 [==============================] - 0s 34us/step - loss: 0.0333 - acc: 0.9833 - val_loss: 0.3094 - val_acc: 0.9048
Epoch 8/40
180/180 [====

In [36]:
# Temporary way of computing true positive rate / false positive rate
for i, word in enumerate(motions):
    pred = model.predict(X_test[(y_test == i).reshape(y_test.shape[0],), :], batch_size=X_test.shape[0])
    pred = pred.argmax(axis=-1)
    print("True '{}' rate: {}".format(word, sum(pred == i)/len(pred)))

True 'nothing' rate: 1.0
True 'unarmed-attack-r3' rate: 0.9230769230769231


In [37]:
result = model.evaluate(X_test, y_test_categorical)
print('')
print('test score: {}'.format(result[1]))

23/23 [==============================] - 0s 81us/step

test score: 0.95652174949646


In [38]:
EXPORT_NN = True
if EXPORT_NN:
    model.save('model/keras/model_{}motions.h5'.format(len(motions)))